In [23]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import smtplib, os
from email import encoders
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
import pyperclip
import datetime

import pyautogui as pag

In [298]:
path = r'https://rc.kbsec.com/report/reportList.able'

driver = webdriver.Chrome('../chromedriver.exe')

driver.get(path)
#로그인
login = driver.find_element_by_css_selector('#unionLoginId')
login.send_keys('kch961203')
password = driver.find_element_by_css_selector('#pw')
password.send_keys('pepsi1602&')
btn = driver.find_element_by_css_selector('#loginBtn')
btn.click()

#데이토 조회 대기 및 조회
time.sleep(5)
reportList = driver.find_element_by_css_selector('#reportList')
li_list = reportList.find_elements_by_css_selector('li')#reportList > 

soup = BeautifulSoup(driver.page_source, 'html')


td= str(datetime.date.today())
td = td.replace('-','.')

td= '2021.01.27'

today_list =[]
buy_hold = []
txt = []
time.sleep(1.5)
for li in soup.select('#reportList > li'):
    b = li.select('i.scr_tx5 > i.r_txt.scr_tx5_2')[0].text
    year, month, day = b.split('.')
    if int(month) < 10:
        month = '0' +month
    if int(day) < 10:
        day = '0' + day
    b= '.'.join([year,month,day])

    if td ==b:
        today_list.append(li)
#     if 'buy' in str(li.select('a')[0]):
#         print(True)
        try:
            buy_hold.append(li.select('i.ibtn')[0].text)
            txt.append(li.select('i.scr_tx1.tcrop')[0].text[:-4])
            print(2)
        except:
            buy_hold.append('')


href_list = []
for li in today_list:
    detail = li.select('a')[0].attrs['href']
    href_list.append('https://rc.kbsec.com/'+detail)


with open(r"C:\Users\82107\Desktop\Workspace\데이터\forMyGit\KB.txt") as f:
    key =f.readlines()

#발송 이메일
fromaddress = key[0].replace('\n','')
pw =  key[1]

#수신 이메일
toaddress = 'kch961203@gmail.com'
text = '{} KB 리서치 보고서2'.format(td)

headlist = []
table ='<strong id="content" style=" color: rgb(249, 88, 96); font-weight: bold; font-size: 28px;">목차</strong>'
# 내용 만들기
total = ''
i= 0
td = td.replace('.','_')

for href, bhn in zip(href_list,buy_hold):
    driver.get(href)
    soup = BeautifulSoup(driver.page_source, 'html')
    head_for_table =soup.select('body > div.wrap > div > div.content.view > div.viewTop > div.top_txt > div')[0].text

    head_for_table = head_for_table+'   ' + bhn 
    headlist.append(r'<div style="margin:5px;" ><a href ="#content%s" style="font-size:15px; color:black;" >'%i +head_for_table+r'</a></div>')
    head = str(soup.select('body > div.wrap > div > div.content.view > div.viewTop > div.top_txt > div')[0])
    a =str(soup.select('body > div.wrap > div > div.content.view > div.viewCon > div.viewCon1 > dl.viewCon1_tx1')[0])
    html_head = head
    html_body = a.replace('dt','strong').replace('\n','''<p>\n</p>''')
    html_head= html_head.replace('span','p')
    html_head= html_head.replace('class="top_txt1"','style="font-size: 15px; color:black;"')
    html_head= html_head.replace('class="top_txt2 fB"','id="content%s" style=" color: rgb(249, 88, 96); font-weight: bold; font-size: 28px;"'%i)
    html_body= html_body.replace('<strong', '<strong style="font-size:23px; font-weight:bold;color:black;"')
    html_body= html_body.replace('<dd', '<dd style=" font-size:18px;color:black;"')
    total = total + html_head+'<hr>'+ html_body +'<p><a href="#content" style="font-size:15px; color:black;" >목차</a></p> <hr>'
    i+=1
total = '<!DOCTYPE html><div style="width:80%; margin-right:100px; margin-left:100px; background-color:#f1efed">' + table+"".join(headlist) + total+'</div></html>'

f = open(r'C:/Users/82107/Desktop/KB/{}/{}.html'.format(td.replace('.','_'),td.replace('.','_')),'w')

f.write(total)
f.close()
print('끝1')




driver.fullscreen_window()
hankyung = r'http://consensus.hankyung.com/apps.analysis/analysis.list'
driver.get(hankyung)
during_ = driver.find_element_by_css_selector('#sdate')
during_.click()
driver.find_element_by_css_selector('body > div.ui-daterangepickercontain > div > ul > li.ui-daterangepicker-1개월.ui-corner-all').click()

for key in txt:
    key= key.strip()
    key= key.split(')')[0]
    key +=')'
    input_ = driver.find_element_by_css_selector('#search_text')
    input_.send_keys(key)
    search = driver.find_element_by_css_selector('#f_search > div > div.hd_top > div.btn_r > a:nth-child(1)')
    search.click()
    tr_list = driver.find_elements_by_css_selector('#contents > div.table_style01 > table > tbody > tr')
    pdf_list = driver.find_elements_by_css_selector('#contents > div.table_style01 > table > tbody > tr> td:nth-child(6) > div > a')
    soup = BeautifulSoup(driver.page_source, 'html')
    pdf_name_tag = soup.select('#contents > div.table_style01 > table > tbody > tr > td.text_l')
    pdf_name_list = []
    for tag in pdf_name_tag:
        if '...' in tag.find('a').text:
            pdf_name_list.append(tag.find('strong').text)
        else:
            pdf_name_list.append(tag.find('a').text)

    if key not in os.listdir(r'C:\Users\82107\Desktop\KB\{}/'.format(td.replace('.','_'))):
        os.makedirs(r'C:\Users\82107\Desktop\KB\{}/{}'.format(td.replace('.','_'),key))
        
    for pdf, name in zip(pdf_list, pdf_name_list):
        time.sleep(1)
        pdf.click()
        driver.switch_to.window(window_name=driver.window_handles[-1])
        time.sleep(2)
        pag.moveTo(x=1828,y=63)
        pag.click()
        time.sleep(1)
        pag.moveTo(x=1029,y=47)
        pag.click()
        pag.press('backspace')
        pyperclip.copy(r'C:/Users/82107/Desktop/KB/{}/{}'.format(td.replace('.','_'),key))
        pag.hotkey('ctrl','v')
        pag.press('enter')
        pag.moveTo(x=354,y=940)
        pag.click()
        name = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘’|\(\)\[\]\<\>`\'…》]', '',name)
        pyperclip.copy(name.replace('\'','').replace(',',''))
        pag.hotkey('ctrl','v')
        pag.press('enter')
        driver.close()
        driver.switch_to.window(window_name=driver.window_handles[-1])
    pag.moveTo(x=1270, y=69)
    pag.click()
    pag.hotkey('ctrl','a')
    pag.press('backspace')
driver.quit()

In [45]:
# 제목 인코딩(한글)
euc_text = text.encode('euc-kr')

# 내용 html로 보내기
part1=MIMEText(total,'html')


msg = MIMEMultipart('alternative')
msg['Subject'] = text
msg['From'] = fromaddress
msg['To'] = toaddress
msg.attach(part1)

mail = smtplib.SMTP('smtp.gmail.com', 587)
mail.starttls()
mail.login(fromaddress, pw)
mail.sendmail(fromaddress,toaddress,msg.as_string())
mail.quit()

(221, b'2.0.0 closing connection i25sm1057255pgb.33 - gsmtp')